In [2]:
import json
import gradio as gr
from openai import OpenAI

# Use Ollama and model llama3.2
MODEL = "llama3.2"
openai = OpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama'  # This value doesn't matter for local Ollama, just required for compatibility
)

# System message that controls the behavior of the assistant
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

# List of ticket prices
ticket_prices = {
    "london": "$799",
    "paris": "$899",
    "tokyo": "$1400",
    "berlin": "$499"
}

# Tool function to get the price of a ticket
def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

# Tool definition, following the format required by the LLM
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": price_function}]

# Handle the tool call when the model asks us to run the function
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city, "price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)

    # Check manually if the user asked for ticket price
    lower_msg = message.lower()
    for city in ticket_prices:
        if city in lower_msg:
            price = get_ticket_price(city)
            return f"The price for a ticket to {city.title()} is {price}."

    return response.choices[0].message.content

# Launch the Gradio chat interface
gr.ChatInterface(fn=chat, type="messages").launch()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for 
Tool get_ticket_price called for Berlin
Tool get_ticket_price called for Berlin
Tool get_ticket_price called for Tokyo
